In [5]:
# mypoll/ORM_Exam.ipynb

## Jupyter Lab에서 Django shell을 실행하기 위한 설정
import os
import django
# 환경변수로 config/settings.py의 위치를 설정
os.environ['DJANGO_SETTINGS_MODULE'] = "config.settings"
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

django.setup()

In [6]:
import sys
print(sys.executable)
!{sys.executable} -m pip show django


c:\Users\Playdata\AppData\Local\miniconda3\envs\lang_env\python.exe
Name: Django
Version: 5.2.4
Summary: A high-level Python web framework that encourages rapid development and clean, pragmatic design.
Home-page: https://www.djangoproject.com/
Author: 
Author-email: Django Software Foundation <foundation@djangoproject.com>
License: BSD-3-Clause
Location: c:\Users\Playdata\AppData\Local\miniconda3\envs\lang_env\Lib\site-packages
Requires: asgiref, sqlparse, tzdata
Required-by: 


In [4]:
!pip install django

  Using cached django-5.2.4-py3-none-any.whl.metadata (4.1 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
Using cached django-5.2.4-py3-none-any.whl (8.3 MB)
Using cached sqlparse-0.5.3-py3-none-any.whl (44 kB)

   ---------------------------------------- 0/2 [sqlparse]
   ---------------------------------------- 0/2 [sqlparse]
   ---------------------------------------- 0/2 [sqlparse]
   ---------------------------------------- 0/2 [sqlparse]
   ---------------------------------------- 0/2 [sqlparse]
   ---------------------------------------- 0/2 [sqlparse]
   ---------------------------------------- 0/2 [sqlparse]
   -------------------- ------------------- 1/2 [django]
   -------------------- ------------------- 1/2 [django]
   -------------------- ------------------- 1/2 [django]
   -------------------- ------------------- 1/2 [django]
   -------------------- ------------------- 1/2 [django]
   -------------------- ------------------- 1/2 [django]
   --------

In [14]:
# 조회 테스트
from polls.models import Question, Choice

q = Question.objects.all()
q

<QuerySet [<Question: 1. 질문1 이름은?>]>

# 조회
- ModelClass.objects -> Model Manager를 반환.
- ModelManager: SQL작업을 할 수있는 메소드들을 제공하는 객체

## 조회메소드
- `all()`: 전체 조회
- `filter()`, `exclude()`: 조건으로 조회(where절)
- `get()`: 조회결과가 하나인 조건으로 조회(PK로 조회)

## 조회결과
- `QuerySet` 객체: 조회결과가 여러개일때 QuerySet에 모아서 반환.
  - 조회결과를 바탕으로 추가 DB 작업을 진행할 수 있는 메소드들을 제공.
  - 개별 데이터는 Model 객체에 담아서 반환.
- Model 객체: 조회결과가 하나(`get()`) 일 때 

In [15]:
from polls.models import Question, Choice

In [16]:
model_manager = Question.objects
type(model_manager)

django.db.models.manager.Manager

In [17]:
result = model_manager.all()
print("조회한 데이터개수:", len(result))
print("all()로 실행된 SQL문을 조회")
print(result.query)

조회한 데이터개수: 1
all()로 실행된 SQL문을 조회
SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [18]:
print(type(result))
result

<class 'django.db.models.query.QuerySet'>


<QuerySet [<Question: 1. 질문1 이름은?>]>

In [19]:
# QuerySet -> Iterable
for q in result:
    # print(type(q))
    print(q, q.pub_date)

1. 질문1 이름은? 2025-07-08 01:06:23.662065+00:00


In [20]:
# QuerySet -> Subscriptable
q = result[1]
print(q.id, q.pk)
print(q.question_text, q.pub_date)
print(type(q.pk), type(q.question_text), type(q.pub_date))

IndexError: list index out of range

In [21]:
# QuerySet - 첫번째, 마지막 index값 조회
q_s = result.first()
q_e = result.last()
q_s.pk, q_e.pk

(1, 1)

In [22]:
# slicing -> 결과: list
s_result = result[:3]
print(type(s_result))
s_result

<class 'list'>


[<Question: 1. 질문1 이름은?>]

In [23]:
# 음수 indexing은 지원하지 않는다.
# result[-2] 

In [24]:
## QuerySet을 이용해서 정렬 (order by) 
##  - QS.order_by("기준Field명"): ASC, QS.order_by("-기준Field명"): DESC
# result.order_by("question_text")
result.order_by("-question_text")

<QuerySet [<Question: 1. 질문1 이름은?>]>

In [25]:
# Choice에 모든 데이터를 조회 -> choice_text 기준으로 정렬
results = Choice.objects.all().order_by("choice_text")
for c in results:
    print(c.pk, c.choice_text, c.votes)

1 이름은 장고 1


In [26]:
# order by choice_text, votes desc
results = Choice.objects.all().order_by("choice_text", "-votes")
for c in results:
    print(c.pk, c.choice_text, c.votes)

1 이름은 장고 1


In [27]:
print(results.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."choice_text" ASC, "polls_choice"."votes" DESC


### Where 절
- filter()
  - 조회조건이 True인 행들을 조회 -> QuerySet을 반환
- exclude()
  - 조회조건이 False인 행들을 조회 -> QuerySet을 반환
- get()
  - 조회조건이 True인 행이 1개일 때 조회.-> Model에 결과를 담아서 반환.
  - 조회결과가 2행 이상이거나 없을 경우 Exception 발생.
- **조회조건**
  - `Field이름__비교연산자 = 비교할 값`

In [28]:
# pk 조회 -> 결과 1행 | 0행
# 동등 비교: field명 = 값
result = Question.objects.get(pk=1)    # where id = 1. 조회결과가 1개
result = Question.objects.filter(pk=1) # where id = 1. 조회결과가 0개 이상
result = Question.objects.exclude(pk=1)# where not id = 1

print(type(result))
print(result)



<class 'django.db.models.query.QuerySet'>
<QuerySet []>


In [29]:
# 비교 연산 
result = Choice.objects.filter(pk__lt=5) # where pk < 5
result = Choice.objects.filter(pk__lte=5) # where pk <= 5
result = Choice.objects.filter(pk__gt=5) # where pk > 5
result = Choice.objects.filter(pk__gte=5) # where pk >= 5
result = Choice.objects.filter(pk=5) # where pk = 5

for r in result:
    print(r)

In [30]:
# 문자열 부분일치 - like ( xxx를 포함, xxx로 시작, xxx로 끝)
result = Question.objects.filter(question_text__contains='색깔')
# question_text like '%색깔%'
result = Question.objects.filter(question_text__endswith='무엇입니까?')
# question_text like "%무엇입니까?"
result = Question.objects.filter(question_text__startswith="싫어하는")
# question_text like "싫어하는%"

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE "polls_question"."question_text" LIKE 싫어하는% ESCAPE '\'


In [64]:
# in  연산
result = Choice.objects.filter(pk__in=[1, 10, 3]) # pk in (1, 10, 3)
result = Choice.objects.exclude(pk__in=[1, 10, 3]) # pk not in (1, 10, 3)

print(result.query)
for r in result:
    print(r)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" IN (1, 10, 3))
2. 파랑
4. 보라
5. 빨강
6. 검정
7. 보라
8. 주황
9. 곰
11. 고양이
12. 호랑이


In [66]:
# between
result = Choice.objects.filter(pk__range=[4, 8])
# pk between 4 and 8
result = Choice.objects.exclude(pk__range=[4, 8])
# pk not between 4 and 8

print(result.query)
for r in result:
    print(r)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" BETWEEN 4 AND 8)
1. 빨강
2. 파랑
3. 노랑
9. 곰
10. 개
11. 고양이
12. 호랑이


#### where 절의 and, or
- `and`: 조건들을 나열
- 각 조건을 `Q()` 함수에 넣고 `|` 로 연결.

In [68]:
result = Question.objects.filter(
    question_text__endswith='무엇입니까?',
    pk__gte=2
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE ("polls_question"."id" >= 2 AND "polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\')
2. 좋아하는 동물은 무엇입니까?


In [72]:
Q(question_text__endswith='무엇입니까?') | Q(pk__gte=3)

<Q: (OR: ('question_text__endswith', '무엇입니까?'), ('pk__gte', 3))>

In [71]:
from django.db.models import Q
result = Question.objects.filter(
    Q(question_text__endswith='무엇입니까?') | Q(pk__gte=3)
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE ("polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\' OR "polls_question"."id" >= 3)
1. 좋아하는 색깔은 무엇입니까?
2. 좋아하는 동물은 무엇입니까?
3. 싫어하는 색깔은 고르세요.
4. 여행가고 싶은 나라는?


In [74]:
# ~Q() : Not
result = Question.objects.filter(
    ~Q(question_text__endswith='무엇입니까?') | Q(pk__gte=4)
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE (NOT ("polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\') OR "polls_question"."id" >= 4)
3. 싫어하는 색깔은 고르세요.
4. 여행가고 싶은 나라는?


#### 조회할 컬럼들을 선택.
- `values(Field명, ...)`
- 개별 조회 결과을 Dictionary로 반환.

In [85]:
result = Question.objects.all().values("pk", "question_text")
result = Choice.objects.filter(pk__lt = 5).values('pk', 'votes')
# select pk, votes from choice where pk < 5
print(result.query)
result
# for r in result:
#     print(type(r), r['pk'], r['question_text'])

SELECT "polls_choice"."id" AS "pk", "polls_choice"."votes" AS "votes" FROM "polls_choice" WHERE "polls_choice"."id" < 5


<QuerySet [{'pk': 1, 'votes': 5}, {'pk': 2, 'votes': 0}, {'pk': 3, 'votes': 0}, {'pk': 4, 'votes': 0}]>

### 집계함수
- `aggregate(집계함수(집계기준field명), 집계함수(field명), ...)`
  - `select avg(salary), count(comm_pct), max(salary) from emp`
- **groupby**
  - values("groupby기준 컬럼").annotate(집계함수)

In [31]:
from django.db.models import (
    Count, # 값의 개수(null 제외한)
    Sum,   # 합계
    Avg,   # 평균
    Min,   # 최소
    Max,   # 최대값
    StdDev, # 표준편차
    Variance # 분산값
)

In [32]:
result = Choice.objects.aggregate(
    Count("votes"),
    Avg("votes"),
    Max("pk"),
    Min("pk"),
    Sum("votes"),
    StdDev("votes"),
    Variance("votes")

)

print(result)
# 반환: dictionary
# default key: field명__집계명

{'votes__count': 1, 'votes__avg': 1.0, 'pk__max': 1, 'pk__min': 1, 'votes__sum': 1, 'votes__stddev': 0.0, 'votes__variance': 0.0}


In [33]:
result = Choice.objects.aggregate(
    cnt=Count("votes"),
    min=Min("votes"),
    max=Max("votes")
)
result

{'cnt': 1, 'min': 1, 'max': 1}

In [34]:
# 집계 결과를 연산
# 변수명 = (집계함수 - 집계함수)
Choice.objects.aggregate(min_max_diff = Max("pk") - Min("pk"))

{'min_max_diff': 0}

In [35]:
######### groupby + 집계
result = Choice.objects.values("question").annotate(
    min=Min("votes"),
    max=Max("votes")
)

In [36]:
result

<QuerySet [{'question': 1, 'min': 1, 'max': 1}]>

In [44]:
c1 = Choice.objects.get(pk=4)
c1.pk, c1.id, c1.choice_text, c1.votes, c1.question

(4, 4, '경기', 5, <Question: 2. 사는 지역은?>)

In [49]:
result_list = Choice.objects.filter(pk__lte=5)

for result in result_list:
    print(result.question.question_text, result.choice_text, result.votes)


질문1 이름은? 이름은 장고 1
질문1 이름은? 2. 이름은 투코 2
사는 지역은? 서울 3
사는 지역은? 경기 5
좋아하는 음료는? 보드카 2


In [51]:
# 부모테이블 기준으로 자식테이블의 데이터(모델)를 조회
# - 부모모델객체. 자식 모델 클래스의이름(소문자).set'을 통해서 부모객체를 
#   참조하는 자식 데이터들을 **조회**할 수 있다.

q1 = Question.objects.get(pk=1)
print(q1.pk, q1.question_text, q1.pub_date)

1 질문1 이름은? 2025-07-08 01:06:23.662065+00:00


In [54]:
q1.choice_set

<django.db.models.fields.related_descriptors.create_reverse_many_to_one_manager.<locals>.RelatedManager at 0x221ef3c5250>

In [56]:
# insert
new_q = Question(question_text="배우고 싶은 언어는 무엇입니까?")
new_q.save()

In [ ]:
# insert후에 자동저장되는 값들이 모델객체의 field에 저장된다.
print(new_q.pk, new_q.question_text, new_q.pub_date)


4 배우고 싶은 언어는 무엇입니까? 2025-07-08 02:22:58.108693+00:00


In [58]:
for q in Question.objects.all():
    print(q.pk, q.question_text, q.pub_date)

1 질문1 이름은? 2025-07-08 01:06:23.662065+00:00
2 사는 지역은? 2025-07-08 01:22:52.284642+00:00
3 좋아하는 음료는? 2025-07-08 01:24:01.634678+00:00
4 배우고 싶은 언어는 무엇입니까? 2025-07-08 02:22:58.108693+00:00


In [ ]:
q = Question.objects.get(pk=4)
q.question_text = "여행 가곳픈곳 정해"

<QuerySet []>

In [60]:
c = Choice(choice_text="미국", votes=10, question=q)
c.save()

In [61]:
c = Choice(choice_text="일본", votes=6, question=q)
c.save()

In [62]:
c = Choice(choice_text="영국", votes=3, question=q)
c.save()

In [63]:
for c in q.choice_set.all():
    print(c.choice_text, c.votes)

미국 10
일본 6
영국 3


In [ ]:
#delete
# - `모델 객체.detele()`

In [ ]:
# 직접 SQL구문 실행
result = Question.objects.raw("select * from polls_choice")
for r in result :
    print(r, r.question)

AttributeError: 'Question' object has no attribute 'question'